In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 999)
df_a = pd.read_csv("ERSA.csv")
df_d = pd.read_csv("ERSD.csv")
df_s = pd.read_csv("ERSS.csv")


In [75]:
from datetime import datetime
df_a["admission_date"] = df_a["ADMISSION_STATUS_DATE"].apply(lambda x: datetime.strptime(x, "%d-%b-%y"))

df_admission = df_a[["EMPLID","admission_date"]]
df_admission.shape

(83694, 2)

In [3]:
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

def get_clean_data(df):
    dfCBA=df[((df['STUDENT_COLLEGE_CODE']=='CBA') | (df['CLASS_COLLEGE_CODE']=='CBA')) & (df['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
    dfCBA = dfCBA.sort_values(by=['EMPLID','CLASS_TERM'])

    df_clean =dfCBA[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
    
    return df_clean

In [4]:
df_dataset_1 =pd.read_csv('dataset-Encrypted-2012-13.csv')
df_dataset_2 =pd.read_csv('dataset2-Encrypted-2012-13.csv')
print(df_dataset_1.shape)
print(df_dataset_2.shape)

df_dataset = pd.concat([df_dataset_1, df_dataset_2])
print(df_dataset.shape)
df1 = get_clean_data(df_dataset)
print(df1.shape)

(256112, 133)
(181396, 133)
(437508, 133)
(49882, 30)


In [5]:
df_encrypted_1=pd.read_csv('encrypted-2009-2012.csv')
df_encrypted_2=pd.read_csv('encrypted-2009-2012-part2.csv')

print(df_encrypted_1.shape)
print(df_encrypted_2.shape)
df_encrypted = pd.concat([df_encrypted_1, df_encrypted_2])
print(df_encrypted.shape)
df2 = get_clean_data(df_encrypted)
print(df2.shape)

(949153, 127)
(31060, 127)
(980213, 127)
(122476, 30)


In [6]:
df_spring = pd.read_csv('Spring2014dataset.csv', encoding="ISO-8859-1")
df = df_spring[((df_spring['STUDENT_COLLEGE_CODE']=='CBA') | (df_spring['CLASS_COLLEGE_CODE']=='CBA')) & (df_spring['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
df3 = df.sort_values(by=['EMPLID','CLASS_TERM'])
df3['EMPLID']= df3['EMPLID'].apply(lambda x: '{0:0>9}'.format(x))
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

#list
df3['EMPLID']=df3['EMPLID'].apply(QBXEncode)
df3=df3[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
print(df3.shape)

(15895, 30)


In [7]:
df_student_lvl=pd.read_csv('SECTION_STUDENT_LVL_DATA-2017-2014.csv')
df4 = get_clean_data(df_student_lvl)
print(df4.shape)

(113787, 30)


In [8]:
dataList = [df1,df2, df3,df4]
for data in dataList:
    print(data.shape)
dfCBA = pd.concat(dataList).sort_values(by=['EMPLID','CLASS_TERM'])
print(dfCBA.shape)

(49882, 30)
(122476, 30)
(15895, 30)
(113787, 30)
(302040, 30)


In [81]:
import numpy as np

#select from 11 classes
cbaClassList = ["Business Communications","Business Finance","Business Statistics I","Business Strategy & Policy","International Business","Legal & Regulatry Env Business","Management Information Systems","Marketing","Organizational Behavior","Princip of Mgmt and Operations","Principles of Management"]
cbaClassCode = ['HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']

dfCBA11 =  dfCBA[dfCBA['CLASS_DESCR'].isin(cbaClassList)]

#select only valid scores: ['A', 'B', 'C', 'D', 'F']
scoreDict = {"A" : 4, "B" : 3, "C" : 2, "D": 1, "F": 0}
scoreList = list(scoreDict.keys())
 
df =  dfCBA11[ dfCBA11['ENRL_OFFICIAL_GRADE'].isin(scoreList)]

majorList = ['Accountancy BS', 'Finance BS', 'Human Resources Management BS', 'International Business BS',  'Management BS',  'Marketing BS',  'Mgmt Info Systems BS', 'Oper Supply Chain Mgmt  BS', 'Operations Management BS']
df_ttest = df[df['STUDENT_ACAD_PLAN'].isin(majorList)]

df_ttest["score"] = df_ttest["ENRL_OFFICIAL_GRADE"].map(scoreDict)
df_ttest["lesson"] = df_ttest["CLASS_I"].apply(lambda x: x[:-3])  

df_ttest['units']= df_ttest['ENRL_UNITS_TAKEN'].apply(lambda x: int(x))
df_gpa = df_ttest.groupby('EMPLID').apply(lambda x: np.average(x["score"], weights = x["units"], axis = 0)).to_frame(name = 'GPA').reset_index()
df_ttest = df_ttest.merge(df_gpa, on = ['EMPLID'], how ='left')

print(df_ttest.shape)
 


(79917, 34)


In [61]:
df_dataset_1.tail()

,REPORT_DATE,CLASS_SESSION_CODE,CLASS_TERM,CTERM_TERM_LDESC,CLASS_SESSION_CENSUS,SESST_ACAD_CAREER,EMPLID,STUDENT_ACAD_CAREER_CODE,STUDENT_ACAD_CAREER,STUDENT_ACAD_PROG_PRIMARY_CODE,STUDENT_ACAD_PROG_PRIM_SDESC,STUDENT_ACAD_PROG_PRIM_LDESC,STUDENT_COLLEGE_APDB,STUDENT_COLLEGE_CODE,STUDENT_COLLEGE,STUDENT_DEPT_APDB,STUDENT_DEPT_CODE,STUDENT_DEPT,STUDENT_SUBDEPT_APDB,STUDENT_SUBDEPT_CODE,STUDENT_SUBDEPT,STUDENT_ACADEMIC_PLAN,STUDENT_ACAD_PLAN_CODE,STUDENT_ACAD_PLAN,STUDENT_SUBACAD_PLAN,CLASS_SID,CLASS_ACAD_CAREER_CODE,CLASS_ACAD_CAREER,CLASS_COLLEGE_APDB,CLASS_COLLEGE_CODE,CLASS_COLLEGE,CLASS_DEPT_APDB,CLASS_DEPT_CODE,CLASS_DEPT,CLASS_SUBDEPT_APDB,CLASS_SUBDEPT_CODE,CLASS_SUBDEPT,CLASS_I,CLASS_II,REMEDIAL_COURSES,REMEDIAL_MATH,REMEDIAL_ENGLISH,CLASS_PREFIX,CLASS_PREFIX_SHORTDESC,CLASS_PREFIX_LONGDESC,CLASS_CATALOG_NBR,CLASS_SECTION,CLASS_DESCR,CLASS_TYPE_LDESC,CLASS_COMPONENT_CD,CLASS_CLASS_STAT_LDESC,CLASS_ENRL_TOT,CLASS_ENRL_CAP,CLASS_FACILITY_SDESC,ONLINE_COURSE,ENRL_UNITS_TAKEN,ENRL_UNITS_PROGRESS,ENRL_UNITS_BILLING,ENRL_GRADING_BASIS,ENRL_OFFICIAL_GRADE,ENRL_GRADE_DATE,ENRL_EARNED_CREDIT,ENRL_UNITS_ATTEMPTED,ENRL_GRADE_POINTS,ENRL_STATUS_CD,ENRL_STATUS_SDESC,ENRL_STATUS_LDESC,ENRL_STATUS_REASON_CD,ENRL_STATUS_REASON_SDESC,ENRL_STATUS_REASON_LDESC,ENRL_ENRL_ADD_DT,ENRL_RQMNT_DESIGNTN,ENRL_RQMNT_DESIGNTN_LDESC,ENRL_ENRL_DROP_DT,ENRL_REPEAT_CODE,ENRL_LAST_DROP_DT_STMP,CTERM_WITHDRAW_LDESC,CTERM_WITHDRAW_REASON_LDESC,CTERM_WITHDRAW_DATE,CLASS_GE_FLAG,CLASS_DIVISION,GPA_FLAG,CATTR_CRSE_ATTR_VALUE_DESCR,CSUFONL_FTES_TARGET,CLASS_INSTRUCTION_MODE,CLASS_START_DATE,CLASS_END_DATE,CLASS_FACILITY_ID,CLASS_FACILITY_BLDG_CD,CLASS_FACILITY_ROOM,CLASS_FACILITY_LDESC,CLASS_FACILITY_TYPE,CLASS_FACILITY_LOCATION,CLASS_WAIT_TOT,CLASS_STDN_MTG_PAT,CLASSM_MEETING_TIME_START,CLASSM_MEETING_TIME_END,CLASSM_MEETING_PAT_DESCR,CLASSM_MONDAY,CLASSM_TUESDAY,CLASSM_WEDNESDAY,CLASSM_THURSDAY,CLASSM_FRIDAY,CLASSM_SATURDAY,CLASSM_SUNDAY,STUDENT_TYPE,APPP_ACAD_PROG_SDESC,APPP_ACAD_PROG_LDESC,STUDENT_TYPE_FINAL,CTERM_ACAD_LEVEL_BOT_CD,CTERM_ACAD_PLAN_PRIMARY_SDESC,RES_EMPLID,RES_ACAD_CAREER,RES_ACAD_CAREER_LDESC,RES_EFFECTIVE_TERM,RES_RESIDENCY_DT,RES_TUITION_RESIDENCY,RES_TUITION_EXCPT,RES_TUITION_EXCPT_LDESC,RES_TUITION_EXCPT_SDESC,MAX_RES_EFFECTIVE_TERM,PERS_EMPLID,PERS_GENDER,PERS_PRIMARY_ETHNICITY,PERS_PRIMARY_ETHNIC_GRP_LDESC,PERS_PRIMARY_ETHNICITY_SDESC,PERS_PRIMARY_ETHNICITY_LDESC,PERS_HOME_COUNTRY_LDESC,PERS_HOME_CITY,PERS_HOME_STATE,PERS_HOME_STATE_DESC,PERS_HOME_POSTAL,ROW_NUMBER
256107,3-Oct-17,1,2124,Fall 2012,24-Sep-12,UGRD,QBX0A9K6Z7X2,UGRD,Undergraduate,UGD,Undergrad,Undergraduate Degree,94 - UNPG,UNPG,University Programs,738 - UNIV,UNIV,University Programs,,,,NDUGOT00U1,NDUGOT00U1,Undeclared Undergraduate,NaN,18669,UGRD,Undergraduate,65 - CNSM,CNSM,Natural Sciences & Mathematics,487 - MATH,MATH,Mathematics,NaN,NaN,NaN,MATH 113 02,MATH 113,Non Remedial Courses,Non Remedial Math,Non Remedial English,MATH,MATH,Mathematics,113,2,Precalculus Algebra,Enrollment Section,SEM,Active,268,277,017-151,Not_Online,3,3,3,GCR,C,20-Dec-12,Y,I,6,E,Enrolled,Enrolled,ENRL,Enrolled,Enrolled,4-Jun-12,103,AREA: B2 - Foundation,NaN,,NaN,Not Withdrawn or Cancelled,NaN,NaN,GE,Lower-Division,Exclude_from_GPA,NaN,Include,P,27-Aug-12,7-Dec-12,017-151,17,151,LH Room 151,LCTR,LB,0,MW,12:30:00,13:45:00,,Y,N,Y,N,N,N,N,NaN,NaN,NaN,Continuing UG,20,NDUGOT00U1,9968762.0,UGRD,Undergraduate,2114.0,20-Sep-11,R,Resident of California,NaN,NaN,2114.0,9968762,F,4,Asian,Asian,Asian/Pacific Islander,United States,Los Gatos,CA,California,95032,256108
256108,3-Oct-17,1,2124,Fall 2012,24-Sep-12,UGRD,QBX0A9K6Z7X2,UGRD,Undergraduate,UGD,Undergrad,Undergraduate Degree,94 - UNPG,UNPG,University Programs,738 - UNIV,UNIV,University Programs,,,,NDUGOT00U1,NDUGOT00U1,Undeclared Undergraduate,NaN,12685,UGRD,Undergraduate,94 - UNPG,UNPG,University Programs,530 - S/I,S/I,Supplemental Instruction,NaN,NaN,NaN,S/I 60 24,S/I 60,Non Remedial Courses,Non Remedial Math,Non Remedial English,S/I,S/I,Supplem

In [59]:
df_s.head()

,RANK,EMPLID,YEARS,TERM,CAMPUS_GPA,TOTAL_GPA,TOTAL_UNITS_EARNED,HS_GRAD_YEAR,TUA_LOWER_DIVISION,TUA_UPPER_DIVISION,TUA_PRE_COLLEGIATE,TUA_GRADUATE,INSTITUTION_ORIGIN_CODE,MAJOR_CODE,GE_CRITICAL_THINKING_STATUS,GE_ENGLISH_COMPOSITION_STATUS,ADMISSION_BASIS_CODE,CONCENTRATION_CODE,TRANSFER_GPA,TRANS_MODEL_CURR_NBR,TRANSFER_UNITS_EARNED,US_MILITARY_STATUS,US_CURRENT_MILITARY_STATUS,US_MILITARY_DEPENDENT_STATUS,PELL_YEAR,PELL_TERM,PELL_OFFER,PELL_DISBURSED
0,1,QBX0A0A1A0C5,2009,4,357,357,690,2007.0,30,90,0,0,600036.0,NaN,N,N,B,5828.0,0.0,NaN,120.0,NaN,NaN,NaN,2009.0,4.0,2366.0,2366.0
1,2,QBX0A0A1A0C5,2012,2,330,330,1340,2007.0,70,70,0,0,600036.0,NaN,N,N,B,5828.0,NaN,NaN,120.0,NaN,NaN,NaN,2012.0,2.0,2775.0,2775.0
2,3,QBX0A0A1A0C5,2011,4,334,334,1190,2007.0,40,110,0,0,600036.0,NaN,N,N,B,5828.0,NaN,NaN,120.0,NaN,NaN,NaN,2011.0,4.0,2775.0,2775.0
3,4,QBX0A0A1A0C5,2011,2,331,331,1060,2007.0,20,110,0,0,600036.0,NaN,N,N,B,5828.0,NaN,NaN,120.0,NaN,NaN,NaN,2011.0,2.0,2775.0,2775.0
4,5,QBX0A0A1A0C5,2010,4,349,349,910,2007.0,70,80,0,0,600036.0,NaN,N,N,B,5828.0,0.0,NaN,120.0,NaN,NaN,NaN,2010.0,4.0,2675.0,2675.0


# Class score description for Management Information Systems

In [11]:
def describe(className):
    df = df_ttest[ df_ttest['CLASS_DESCR']== className  ]
    return df.groupby(["STUDENT_ACAD_PLAN"])[["score"]].describe()

In [12]:
d = describe("Management Information Systems")
d

score                                          \
                                count      mean       std  min  25%  50%  75%   
STUDENT_ACAD_PLAN                                                               
Accountancy BS                 1781.0  2.824818  0.848443  0.0  2.0  3.0  3.0   
Finance BS                     1265.0  2.732016  0.848562  0.0  2.0  3.0  3.0   
Human Resources Management BS   361.0  2.645429  0.810718  0.0  2.0  3.0  3.0   
International Business BS       449.0  2.652561  0.834255  0.0  2.0  3.0  3.0   
Management BS                  1434.0  2.638773  0.847345  0.0  2.0  3.0  3.0   
Marketing BS                   1401.0  2.659529  0.815298  0.0  2.0  3.0  3.0   
Mgmt Info Systems BS            433.0  2.854503  0.852330  0.0  2.0  3.0  3.0   
Oper Supply Chain Mgmt  BS       85.0  2.858824  0.847306  0.0  2.0  3.0  3.0   
Operations Management BS         11.0  3.090909  0.831209  2.0  2.5  3.0  4.0   

                                    
                               max  
STUDENT_ACAD_PLAN                   
Accountancy BS                 4.0  
Finance BS                     4.0  
Human Resources Management BS  4.0  
International Business BS      4.0  
Management BS                  4.0  
Marketing BS                   4.0  
Mgmt Info Systems BS           4.0  
Oper Supply Chain Mgmt  BS     4.0  
Operations Management BS       4.0

# Hypothesis1 : Higher score for major student

In [13]:
import scipy 
import scipy.stats
import numpy as np

majorList.sort()

def run_ttest_on_class_major(classCode, major):
    df = df_ttest[ df_ttest['lesson']== classCode  ]
    group1 = df[ df['STUDENT_ACAD_PLAN'] == major ]
    group2 = df[ df['STUDENT_ACAD_PLAN'] != major ]
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())

majorClassList = [('Human Resources Management BS', 'HRM 360'), ('Management BS', 'MGMT 300'), ('Marketing BS', 'MKTG 300'), \
                  ('International Business BS','CBA 300'), ('Mgmt Info Systems BS', 'I S 300'), ('Mgmt Info Systems BS', 'I S 301'), ('Finance BS', 'FIN 300')]
     
t_test_results = {}
scores = {}
for  majorName, className in majorClassList:
    t_test_results[className + " : " + majorName], scores[className + " : " + majorName ] = run_ttest_on_class_major(className, majorName)

#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorScore'] = scores_df[0].to_numpy()
results_df['nonMajorScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,majorScore,nonMajorScore
HRM 360 : Human Resources Management BS,-0.455351,6.488684e-01,2.557292,2.580923
MGMT 300 : Management BS,-4.078622,4.571056e-05,2.564351,2.660422
MKTG 300 : Marketing BS,-0.432440,6.654329e-01,2.642953,2.653528
CBA 300 : International Business BS,-0.701952,4.827293e-01,2.444444,2.473967
I S 300 : Mgmt Info Systems BS,3.364837,7.698818e-04,2.854503,2.714012
I S 301 : Mgmt Info Systems BS,-1.074764,2.825129e-01,2.623404,2.659043
FIN 300 : Finance BS,5.078887,3.880284e-07,2.851661,2.693636


# Hypothesis 2: Different class time has score difference

In [14]:
df_ttest["hour"] = df_ttest["CLASSM_MEETING_TIME_START"].str.split(":").str[0]
df_ttest['hour'] = df_ttest['hour'].replace(np.nan, 0)

df_ttest["morning"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 12 else  0)
df_ttest["afternoon"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 12 and int(x) < 18 else  0)
df_ttest["night"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 18  else  0)

def run_ttest_on_day_time(dayTime):
    group1 = df_ttest[ df_ttest[dayTime] == 1 ]
    group2 = df_ttest[ df_ttest[dayTime] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
 
t_test_results = {}
scores = {}
for dayTime in ["morning", "afternoon", "night"]:
    t_test_results[dayTime], scores[dayTime] = run_ttest_on_day_time(dayTime)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inPeriodScore'] = scores_df[0].to_numpy()
results_df['outPeriodScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,inPeriodScore,outPeriodScore
morning,-0.07468,0.940470,2.668255,2.668778
afternoon,1.12537,0.260436,2.672453,2.665179
night,-1.26825,0.204712,2.660967,2.670808


# Hypothesis 3: Students take 2 or more quantitative classes has lower score

In [104]:
quantitativeList = ['FIN 300', 'I S 300', 'I S 301']
df_ttest["quantitative"] = df_ttest['lesson'].apply(lambda x: 1 if x in quantitativeList else  0)
#df_ttest.head(10)

df2 = df_ttest.groupby(['EMPLID', 'CTERM_TERM_LDESC'])['quantitative'].sum().reset_index().rename(columns={'quantitative':'sum_quantitative'})
df_quant = df_ttest.merge(df2, on = ['EMPLID','CTERM_TERM_LDESC'], how ='left')

#print(df_ttest.shape)
#print(df_quant.shape)

def run_ttest_on_quant():
    group1 = df_quant[ df_quant["sum_quantitative"] >= 2 ]
    group2 = df_quant[ df_quant["sum_quantitative"] <  2 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
 
t_test_results = {}
scores = {}
t_test_results["Hypothesis3"], scores["Hypothesis3"] = run_ttest_on_quant()
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_df['MoreQuantClassScore'] = scores_df[0].to_numpy()
results_df['LessQuantClassScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,MoreQuantClassScore,LessQuantClassScore
Hypothesis3,6.351388,2.145225e-10,2.710655,2.658629


# Hypothesis 4: Which Major has higher GPA?

In [16]:
df_ttest["major_mis"] = df_ttest["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Mgmt Info Systems BS' else  0) 
df_ttest["major_fin"] = df_ttest["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Finance BS' else  0) 
df_ttest["major_acc"] = df_ttest["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Accountancy BS' else  0) 
df_ttest["major_mrk"] = df_ttest["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Marketing BS' else  0) 

def run_ttest_on_major(major):
    group1 = df_ttest[ df_ttest[major] == 1 ]
    group2 = df_ttest[ df_ttest[major] == 0 ]
    
    return scipy.stats.ttest_ind(group1['GPA'], group2['GPA']), (group1["GPA"].mean(), group2["GPA"].mean())
 
t_test_results = {}
scores = {}
for major in ["major_mis", "major_fin", "major_acc", "major_mrk"]:
    t_test_results[major], scores[major] = run_ttest_on_major(major)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorGPA'] = scores_df[0].to_numpy()
results_df['otherMajorGPA'] = scores_df[1].to_numpy()
results_df


,statistic,pvalue,majorGPA,otherMajorGPA
major_mis,-2.858154,4.262237e-03,2.642785,2.670214
major_fin,7.912255,2.559830e-15,2.707294,2.660456
major_acc,34.114176,3.042385e-253,2.801828,2.625163
major_mrk,-16.697078,1.757368e-62,2.591954,2.686907


# Hypothesis 5: Students in junior year taking quantitative class have lower score

In [126]:
from datetime import timedelta
    
df_5 = df_quant.merge(df_admission, on = ['EMPLID'], how ='left')

df_5["course_date"] = df_5["CTERM_TERM_LDESC"].apply(lambda x: datetime.strptime(x.replace("Fall","09 01").replace("Spring","01 01"), "%m %d %Y"))
df_5["start_time"]  = df_5.course_date - df_5.admission_date
df_5["junior_year"] = df_5["start_time"].apply(lambda x: 1 if x >  timedelta(days=365*2) else 0)

df_5 = df_5.loc[df_5["admission_date"].notnull()]
df_5 = df_5.loc[df_5["quantitative"] == 1]

print(df_5.shape)
#print(df_5[["CTERM_TERM_LDESC", "course_date", "admission_date", "start_time", "junior_year", "quantitative"]])

def run_ttest_on_h5(df, check_gpa = False):
    group1 = df[ df["junior_year"] == 1 ]
    group2 = df[ df["junior_year"] == 0 ]
    
    if check_gpa:
        return scipy.stats.ttest_ind(group1['GPA'], group2['GPA']), (group1["GPA"].mean(), group2["GPA"].mean())
    else:
        return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
    
    
t_test_results = {}
scores = {}
t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5)
results_gpa = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_gpa['Junior'] = scores_df[0].to_numpy()
results_gpa['Senior'] = scores_df[1].to_numpy()

def run_hypothesis5(lesson):
    t_test_results = {}
    scores = {}
    if lesson == "GPA":
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5, True)
    else:
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5.loc[df_5["lesson"] == lesson])
 
    results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
    scores_df = pd.DataFrame.from_dict(scores, orient='Index')
    results_df['Junior'] = scores_df[0].to_numpy()
    results_df['Senior'] = scores_df[1].to_numpy()
    results_df['Class Name'] = lesson
    return results_df
    
df_hp5_gpa = run_hypothesis5("GPA")
df_hp5_fin300 = run_hypothesis5("FIN 300")
df_hp5_is300 = run_hypothesis5("I S 300")
df_hp5_is301 = run_hypothesis5("I S 301")

df_dataset = pd.concat([df_hp5_gpa, df_hp5_fin300, df_hp5_is300, df_hp5_is301])
df_dataset

(5359, 40)


,statistic,pvalue,Junior,Senior,Class Name
Hypothesis5,-7.359891,2.119330e-13,2.698736,2.966794,GPA
Hypothesis5,-3.696094,2.253922e-04,2.665299,3.016129,FIN 300
Hypothesis5,0.157474,8.748906e-01,2.754975,2.738462,I S 300
Hypothesis5,-2.968016,3.035694e-03,2.689733,2.892473,I S 301


In [17]:
def run_ttest_on_class(classCode):
    df = df_ttest[ df_ttest['lesson']== classCode  ]
    
    #print(majorList)
    t_test_results = {}
    for major1 in majorList:
        for major2 in majorList:
            if major1 >= major2:
                continue

            group1 = df[ df['STUDENT_ACAD_PLAN']== major1]
            group2 = df[ df['STUDENT_ACAD_PLAN']== major2]

            t_test_results[major1 + " : " + major2] = scipy.stats.ttest_ind(group1['score'],group2['score'])

    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    results_df.sort_values(by=['pvalue'])
    return results_df


def class_major_ttest(className, major):
    df = df_ttest[ df_ttest['CLASS_DESCR']== className  ]
    
    #print(majorList)
    t_test_results = {}

    group1 = df[ df['STUDENT_ACAD_PLAN']== major]
    group2 = df[ df['STUDENT_ACAD_PLAN']!= major]

    return scipy.stats.ttest_ind(group1['score'], group2['score'])
    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    results_df.sort_values(by=['pvalue'])
    return results_df
    
t_test_results = {}
for major in majorList:
    for className in cbaClassList:
        df = df_ttest[ df_ttest['CLASS_DESCR']== className  ]

        group1 = df[ df['STUDENT_ACAD_PLAN']== major]
        group2 = df[ df['STUDENT_ACAD_PLAN']!= major]
    
        #print(major, className)
        t_test_results[major+" : " + className] =   scipy.stats.ttest_ind(group1['score'],group2['score'])
  
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']

 
results_df = results_df[ results_df['pvalue'] <= 0.01]  
results_df = results_df.sort_values(by=['pvalue'], ascending=True)
results_df

,statistic,pvalue
Accountancy BS : Business Finance,14.541179,2.575018e-47
Accountancy BS : Business Statistics I,11.123943,1.572335e-28
Marketing BS : Business Finance,-9.038473,1.954846e-19
Accountancy BS : International Business,8.642492,6.540048e-18
Management BS : Business Finance,-7.490529,7.574066e-14
Accountancy BS : Organizational Behavior,7.110061,1.256525e-12
Marketing BS : International Business,-6.250383,4.298494e-10
Management BS : Business Statistics I,-6.060265,1.422195e-09
Accountancy BS : Management Information Systems,5.919251,3.382221e-09
Accountancy BS : Princip of Mgmt and Operations,5.608884,2.135242e-08


# Unsupvised Learning: K means Clusting on GPA 

In [49]:
from sklearn.cluster import KMeans

df_ttest.is_copy = None
df_ttest["mon"] = df_ttest["CLASSM_MONDAY"].apply(lambda x: 1 if x == 'Y' else  0)
df_ttest["tus"] = df_ttest["CLASSM_TUESDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["wed"] = df_ttest["CLASSM_WEDNESDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["thur"]= df_ttest["CLASSM_THURSDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["fri"] = df_ttest["CLASSM_FRIDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["sat"] = df_ttest["CLASSM_SATURDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["sun"] = df_ttest["CLASSM_SUNDAY"].apply(lambda x: 1 if x == 'Y' else  0) 


X = np.asarray(df_ttest["GPA"])
kmeans = KMeans(n_clusters=5).fit(X.reshape(-1,1))  
# Save the labels
df_ttest.loc[:,'GPA_cluster'] = kmeans.labels_

#print(df_ttest.head(1))
#print(kmeans.cluster_centers_)
print(df_ttest[["GPA"]].groupby(df_ttest["GPA_cluster"]).agg(['mean', 'count']))

#print(df_ttest[["GPA"]].groupby(df_ttest["GPA_cluster"]).agg(['mean', 'count']))



                  GPA       
                 mean  count
GPA_cluster                 
0            2.978690  22572
1            2.028935  16347
2            3.586625  14355
3            1.333333   4536
4            2.502918  22107
[[2.97869041]
 [2.02893497]
 [3.58662487]
 [1.33333333]
 [2.50291763]]


In [544]:
grouped = df_ttest['score'].groupby(df_ttest['EMPLID']).mean()
grouped.index.name = 'index'
 
df2 = grouped.to_frame() #pd.DataFrame({'EMPLID':grouped.index, 'mean':grouped.values})
df2.rename(columns = {'score':'avgGPA'}, inplace = True) 
df2['EMPLID'] = list(grouped.index)

df_ttest = df_ttest.merge(df2, on='EMPLID', how='left')
df_ttest.shape

(79917, 32)

In [558]:
df_ttest.is_copy = None
df_ttest["mon"] = df_ttest["CLASSM_MONDAY"].apply(lambda x: 1 if x == 'Y' else  0)
df_ttest["tus"] = df_ttest["CLASSM_TUESDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["wed"] = df_ttest["CLASSM_WEDNESDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["thur"]= df_ttest["CLASSM_THURSDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["fri"] = df_ttest["CLASSM_FRIDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["sat"] = df_ttest["CLASSM_SATURDAY"].apply(lambda x: 1 if x == 'Y' else  0) 
df_ttest["sun"] = df_ttest["CLASSM_SUNDAY"].apply(lambda x: 1 if x == 'Y' else  0) 


df_ttest.head()

/Users/Xuan/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5079: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/Users/Xuan/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5080: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


,CTERM_TERM_LDESC,CLASS_TERM,EMPLID,STUDENT_ACAD_CAREER,STUDENT_ACAD_PROG_PRIM_LDESC,STUDENT_COLLEGE_CODE,STUDENT_DEPT_CODE,STUDENT_ACAD_PLAN,STUDENT_SUBACAD_PLAN,CLASS_SID,CLASS_COLLEGE_CODE,CLASS_DEPT,CLASS_SUBDEPT,CLASS_I,CLASS_DESCR,ONLINE_COURSE,ENRL_UNITS_TAKEN,ENRL_GRADING_BASIS,ENRL_OFFICIAL_GRADE,CLASS_STDN_MTG_PAT,CLASSM_MEETING_TIME_START,CLASSM_MONDAY,CLASSM_TUESDAY,CLASSM_WEDNESDAY,CLASSM_THURSDAY,CLASSM_FRIDAY,CLASSM_SATURDAY,CLASSM_SUNDAY,STUDENT_TYPE_FINAL,RES_TUITION_RESIDENCY,score,avgGPA,mon,tus,wed,thur,fri,sat,sun,major_mis,major_fin,major_acc,major_mrk,lesson
0,Fall 2009,2094,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,NaN,24209,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,MW,15:30:00,Y,N,Y,N,N,N,N,Returning UG,R,2,2.0,1,0,1,0,0,0,0,0,0,0,0,HRM 360
1,Spring 2011,2112,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,OTHLANGU1,29574,CBA,Marketing,NaN,MKTG 300 02,Marketing,Not_Online,3,GCR,C,M,19:00:00,Y,N,N,N,N,N,N,Continuing UG,R,2,2.0,1,0,0,0,0,0,0,0,0,0,0,MKTG 300
2,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,35832,CBA,Management,NaN,MGMT 425 12,Business Strategy & Policy,Not_Online,3,GRD,B,TU,19:00:00,N,Y,N,N,N,N,N,Continuing UG,R,3,2.0,0,1,0,0,0,0,0,0,1,0,0,MGMT 425
3,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,36626,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,TTH,17:30:00,N,Y,N,Y,N,N,N,Continuing UG,R,2,2.0,0,1,0,1,0,0,0,0,1,0,0,HRM 360
4,Fall 2009,2094,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,29153,CBA,Information Systems,NaN,I S 310 11,Business Statistics I,Not_Online,3,GRD,D,TH,19:00:00,N,N,N,Y,N,N,N,Continuing UG,R,1,2.0,0,0,0,1,0,0,0,0,1,0,0,I S 310


In [556]:
df_ttest["CLASS_I"].unique()

array(['HRM 360 08', 'MKTG 300 02', 'MGMT 425 12', 'I S 310 11',
       'MGMT 425 18', 'MGMT 300 05', 'FIN 300 04', 'I S 310 08',
       'BLAW 320 10', 'MGMT 425 04', 'I S 300 09', 'HRM 360 01',
       'MGMT 300 02', 'MGMT 425 11', 'MGMT 425 14', 'CBA 300 06',
       'I S 310 09', 'HRM 360 03', 'MKTG 300 01', 'FIN 300 05',
       'I S 310 06', 'MKTG 300 06', 'MGMT 300 03', 'I S 301 12',
       'BLAW 320 09', 'I S 300 04', 'MGMT 425 17', 'MGMT 425 15',
       'I S 300 06', 'HRM 360 02', 'I S 301 21', 'FIN 300 06',
       'MKTG 300 09', 'I S 310 04', 'I S 300 05', 'I S 301 14',
       'MGMT 425 05', 'MGMT 300 01', 'I S 310 01', 'MGMT 425 16',
       'MGMT 425 19', 'BLAW 320 11', 'CBA 300 02', 'BLAW 320 12',
       'MGMT 425 13', 'I S 301 17', 'FIN 300 03', 'CBA 300 01',
       'I S 300 02', 'MGMT 300 06', 'BLAW 320 03', 'I S 301 03',
       'I S 310 12', 'MGMT 425 07', 'MGMT 425 06', 'BLAW 320 08',
       'I S 310 02', 'MGMT 300 11', 'MGMT 425 01', 'BLAW 320 01',
       'MGMT 425 20', 'I

In [567]:
df_ttest[["lesson", "CLASS_DESCR"]] 

,lesson,CLASS_DESCR
0,HRM 360,Organizational Behavior
1,MKTG 300,Marketing
2,MGMT 425,Business Strategy & Policy
3,HRM 360,Organizational Behavior
4,I S 310,Business Statistics I
5,MGMT 425,Business Strategy & Policy
6,MGMT 300,Principles of Management
7,FIN 300,Business Finance
8,I S 310,Business Statistics I
9,BLAW 320,Legal & Regulatry Env Business


In [557]:
df_ttest["CLASS_DESCR"].unique()     

array(['Organizational Behavior', 'Marketing',
       'Business Strategy & Policy', 'Business Statistics I',
       'Principles of Management', 'Business Finance',
       'Legal & Regulatry Env Business', 'Management Information Systems',
       'International Business', 'Princip of Mgmt and Operations',
       'Business Communications'], dtype=object)

In [570]:
for cls in cbaClassList:
    df = df_ttest[df_ttest["CLASS_DESCR"] == cls]["lesson"]
    print(cls, df.head(1))

Business Communications 25    I S 301
Name: lesson, dtype: object
Business Finance 7    FIN 300
Name: lesson, dtype: object
Business Statistics I 4    I S 310
Name: lesson, dtype: object
Business Strategy & Policy 2    MGMT 425
Name: lesson, dtype: object
International Business 17    CBA 300
Name: lesson, dtype: object
Legal & Regulatry Env Business 9    BLAW 320
Name: lesson, dtype: object
Management Information Systems 11    I S 300
Name: lesson, dtype: object
Marketing 1    MKTG 300
Name: lesson, dtype: object
Organizational Behavior 0    HRM 360
Name: lesson, dtype: object
Princip of Mgmt and Operations 24    MGMT 300
Name: lesson, dtype: object
Principles of Management 6    MGMT 300
Name: lesson, dtype: object


In [568]:
df_ttest["lesson"].unique()   

array(['HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300',
       'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301'],
      dtype=object)

In [547]:
df_ttest["CLASSM_MEETING_TIME_START"].unique()

array(['15:30:00', '19:00:00', '17:30:00', '9:30:00', '16:00:00',
       '17:00:00', '14:00:00', '11:00:00', '12:30:00', '9:00:00',
       '10:00:00', '8:00:00', '09:30:00', '08:00:00', nan, '12:00:00',
       '13:00:00', '09:00:00'], dtype=object)

In [549]:
df_mis = df_ttest[ df_ttest['CLASS_DESCR']== "Management Information Systems"  ]
df_mis.shape

(7220, 43)

In [399]:
import statsmodels.api as sm

mod = sm.OLS(y,X)

fii = mod.fit()

p_values = fii.summary2().tables[1]['P>|t|']

/Users/Xuan/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1755: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/Xuan/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/Xuan/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/Xuan/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/Xuan/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [550]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()



X = df_mis[['mon', 'tus', 'wed', 'thur', 'fri', 'sat', 'sun', 'major_mis', 'avgGPA']]
y = df_mis['score']
regressor.fit(X, y)
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient' ])
coeff_df

,Coefficient
mon,-1.928518e-01
tus,-1.405428e-01
wed,-1.427444e-01
thur,-1.994722e-01
fri,3.330669e-16
sat,0.000000e+00
sun,0.000000e+00
major_mis,1.633365e-01
avgGPA,9.320573e-01


In [554]:
# For statistics. Requires statsmodels 5.0 or more
from statsmodels.formula.api import ols


# Fit the model
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())



                                 OLS Regression Results                                
Dep. Variable:                  score   R-squared (uncentered):                   0.954
Model:                            OLS   Adj. R-squared (uncentered):              0.954
Method:                 Least Squares   F-statistic:                          2.474e+04
Date:                Sun, 28 Jun 2020   Prob (F-statistic):                        0.00
Time:                        19:33:26   Log-Likelihood:                         -6718.0
No. Observations:                7220   AIC:                                  1.345e+04
Df Residuals:                    7214   BIC:                                  1.349e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [555]:
# Analysis of Variance (ANOVA) on linear models
from statsmodels.stats.anova import anova_lm

table = anova_lm(results, typ=2)
table

AttributeError: 'PandasData' object has no attribute 'design_info'